#### Imports

In [1]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
import gc
import pickle
from datetime import datetime
from os import makedirs
from os.path import dirname, join
from pathlib import Path

import numpy as np
import scipy
import yaml
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import KFold

#### Utils

In [3]:
config = {
    "output_dir": "/scratch/st-jiaruid-1/shenoy/projects/scRNA-competition/output/krr-rbf-exp",
    "paths": {
      "x": "/scratch/st-jiaruid-1/shenoy/svd-comp/train_input_cite_svd128.pkl",
      "y": "/arc/project/st-jiaruid-1/yinian/multiome/sparse-data/train_cite_targets_values.sparse.npz",
      "x_test": "/scratch/st-jiaruid-1/shenoy/svd-comp/test_input_cite_svd128.pkl"
    },
    "seed": 42
}

In [4]:
# Load Data
%time
x_train_transformed = pickle.load(open(config["paths"]["x"], "rb"))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [5]:
%time
x_test_transformed = pickle.load(open(config["paths"]["x_test"], "rb"))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


In [6]:
%time
y = scipy.sparse.load_npz(config["paths"]["y"])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs


In [7]:
pca_y = TruncatedSVD(
    n_components=140,
    random_state=config['seed'],
)
y_transformed = pca_y.fit_transform(y)

In [8]:
from catboost import CatBoostRegressor, metrics, MultiTargetCustomMetric

In [9]:
class PCCCatBoostMetric(MultiTargetCustomMetric):
    def is_max_optimal(self):
        # Returns whether great values of metric are better
        return True

    def evaluate(self, approxes, target, weight):
        # approxes is a list of indexed containers
        # (containers with only __len__ and __getitem__ defined),
        # one container per approx dimension.
        # Each container contains floats.
        # weight is a one dimensional indexed container.
        # target is a one dimensional indexed container.

        # weight parameter can be None.
        # Returns pair (error, weights sum)
        y_true, y_pred = np.stack(target, 1), np.stack(approxes, 1)
        corrsum = 0
        for i in range(len(y_true)):
            corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
        return corrsum, y_true.shape[0]

    def get_final_error(self, error, weight):
        # Returns final value of metric based on error and weight
        return error / weight

In [19]:
model = CatBoostRegressor(
    iterations=20,
    loss_function=metrics.MultiRMSE(),
    random_seed=config["seed"],
    task_type='GPU',
#     eval_metric=PCCCatBoostMetric()
)

In [20]:
n = x_train_transformed.shape[0]
frac = 0.8
train_len = int(n * frac)
x_train, y_train = x_train_transformed[:train_len, :], y_transformed[:train_len, :]
x_val, y_val = x_train_transformed[train_len:, :], y_transformed[train_len:, :]

In [21]:
print (x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(56790, 128) (56790, 140) (14198, 128) (14198, 140)


In [28]:
import catboost

In [29]:
catboost.__version__

'1.1'

In [26]:
%time

model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=10, verbose=True)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


CatBoostError: catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'